# Trading signals: Moving Average (MA)

## What is a Moving Average ?
The moving average (MA) is a simple technical analysis tool that smooths out price data by creating a constantly updated average price. The average is taken over a specific period of time, like 10 days, 20 minutes, 30 weeks or any time period the trader chooses. 

Moving average strategies are also popular and can be tailored to any time frame, suiting both long-term investors and short-term traders.

## Why Use a Moving Average ? 
A moving average helps cut down the amount of "noise" on a price chart. Look at the direction of the moving average to get a basic idea of which way the price is moving. If it is angled up, the price is moving up (or was recently) overall; angled down, and the price is moving down overall; moving sideways, and the price is likely in a range.

## Trading Strategies—Crossover

The most relevant crossover trading strategy consiste in applying two moving averages to a price time serie: one longer and one shorter. 

When the shorter-term MA crosses above the longer-term MA, it's a buy signal, as it indicates that the trend is shifting up. This is known as a "golden cross."

Meanwhile, when the shorter-term MA crosses below the longer-term MA, it's a sell signal, as it indicates that the trend is shifting down. This is known as a "dead/death cross."

In [1]:
import pandas as pd
from ipywidgets import ToggleButtons, VBox, HTML
import bqplot as bqp
import numpy as np
from ipywidgets import SelectMultiple, SelectionRangeSlider, HTML, GridBox
from itertools import cycle

In [2]:
dataframe = pd.read_csv('../data/TSLA.csv')
dataframe.set_index('Date', inplace=True)
dataframe.drop('Volume',axis=1,inplace=True)

In [3]:
dataframe.head()

,Open,High,Low,Close,Adj Close
Date,,,,,
2010-06-29,19.000000,25.00,17.540001,23.889999,23.889999
2010-06-30,25.790001,30.42,23.299999,23.830000,23.830000
2010-07-01,25.000000,25.92,20.270000,21.959999,21.959999
2010-07-02,23.000000,23.10,18.709999,19.200001,19.200001
2010-07-06,20.000000,20.00,15.830000,16.110001,16.110001


Generation of random data and plot creation

In [11]:

# Generate some random data to work with
np.random.seed(5)
dates = pd.date_range(end='2019-01-01', periods=120, freq='M')
columns = ['Security 1', 'Security 2', 'Security 3', 'Security 4']

dataframe = pd.DataFrame(np.random.randn(120,4),
                         index=dates,
                         columns=columns).cumsum() / 100

# Define colors to be usedGe
colors = ['#1B84ED', '#CF7DFF', '#FF5A00', '#00D3D6']

# Create scales
scale_x = bqp.DateScale()
scale_y = bqp.LinearScale()

# Create Lines mark
mark_line = bqp.Lines(x=dataframe.index,
                      y=[dataframe[col] for col in dataframe.columns],
                      scales={'x': scale_x, 'y': scale_y},
                      colors=colors,
                      labels=['Security 1', 'Security 2',
                              'Security 3', 'Security 4'],
                      display_legend=True)

# Create Scatter mark to display tooltips
scatter_df = pd.melt(dataframe.reset_index(),
                     id_vars=['index'],
                     var_name='column')
mark_scatter = bqp.Scatter(x=scatter_df['index'],
                           y=scatter_df['value'],
                           skew=scatter_df['column'],
                           scales={'x': scale_x, 'y': scale_y},
                           # Stop points from being visible
                           default_opacities=[0],
                           tooltip=bqp.Tooltip(fields=['y', 'skew', 'x'],
                                               formats=['.3%', '',
                                                        '%m/%d/%Y'],
                                               show_labels=False))

# Create Axes
axis_x = bqp.Axis(scale=scale_x)
axis_y = bqp.Axis(scale=scale_y,
                  orientation='vertical',
                  label='Returns',
                  side='right',
                  tick_format='.2%',
                  label_offset='4em')

# Create Figure
figure_main = bqp.Figure(marks=[mark_line, mark_scatter],
                         axes=[axis_x, axis_y],
                         title='Time series data',
                         layout={'width':'99%',
                                 'height':'450px',
                                 'grid_area':'plot'},
                         title_style={'font-size': '20px'},
                         legend_location='top-left',
                         legend_style={'stroke': 'none'},
                         animation_duration=500,
                         fig_margin={'top': 60, 'bottom': 30,
                                     'left': 10, 'right': 80})

# Create widgets
widget_fieldselect = SelectMultiple(options=dataframe.columns,
                                    layout={'grid_area': 'columns'})
widget_rangeselect = SelectionRangeSlider(options=dataframe.index,
                                          readout=False,
                                          value=[dataframe.index[0],
                                                 dataframe.index[-1]],
                                          layout={'width':'99%',
                                                  'grid_area':'range'})

widget_html = HTML('', layout={'grid_area': 'display'})

# Create color map, so that columns will always be the same color
color_cycle = cycle(colors)
color_map = {col:next(color_cycle) for col in dataframe.columns}

# Define callback functions for the widgets
def update_range(evt):
    min_val, max_val = evt['new']
    scale_x.min = min_val
    scale_x.max = max_val
    widget_html.value = ('<center>{:%Y-%m-%d} - '
                         '{:%Y-%m-%d}</center>').format(min_val, max_val)

def update_columns(evt):
    new_columns = evt['new']
    # Update Lines
    mark_line.y = [dataframe[col] for col in new_columns]
    mark_line.labels = new_columns
    mark_line.colors = [color_map[col] for col in new_columns]

    # Update Scatter
    new_dataframe = scatter_df[scatter_df['column'].isin(new_columns)]
    mark_scatter.x = new_dataframe['index']
    mark_scatter.y = new_dataframe['value']
    mark_scatter.skew = new_dataframe['column']

# Bind callbacks to the widgets
widget_rangeselect.observe(update_range, names=['value'])
widget_fieldselect.observe(update_columns, names=['value'])


# Create GridBox layout
grid = GridBox(children=[figure_main, widget_fieldselect,
                         widget_html, widget_rangeselect],
               layout={'width': '100%',
                       'grid_template_rows': 'auto auto auto',
                       'grid_gap': '5px',
                       'overflow_x': 'hidden',
                       'grid_template_columns': 'min-content auto',
                       'grid_template_areas': '''
                           "plot plot"
                           "columns range"
                           "columns display"'''})
# Display the visualizaiton
grid


GridBox(children=(Figure(animation_duration=500, axes=[Axis(scale=DateScale()), Axis(label='Returns', label_of…